<a href="https://colab.research.google.com/github/gatherheart/Classification_With_BERT/blob/main/news_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kr53hsl2
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kr53hsl2
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=b9f18921712d516bdab61bd445251ab5765b92c99506171f3ef0111ffba14010
  Stored in directory: /tmp/pip-ephem-wheel-cache-hs4cp3vv/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
BASE_PATH = '/content/drive/MyDrive/hyun_data'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Dec 17 15:50:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df = pd.read_csv(BASE_PATH+'/preprocessed_norm.csv', index_col=[0])

In [ ]:
labels_csv = pd.read_csv(BASE_PATH+'/labels_dict2.csv', index_col=[0])

In [ ]:
labels_csv

,문화>방송_연예,국제>중국,사회>사회일반,정치>국회_정당,사회>교육_시험,IT_과학>모바일,정치>행정_자치,사회>사건_사고,문화>문화일반,정치>청와대,사회>여성,국제>유럽_EU,경제>자원,국제>미국_북미,사회>날씨,사회>의료_건강,IT_과학>콘텐츠,경제>증권_증시,정치>선거,국제>일본,사회>미디어,문화>음악,국제>중남미,국제>중동_아프리카,경제>유통,문화>출판,경제>자동차,사회>환경,경제>금융_재테크,사회>장애인,문화>전시_공연,경제>부동산,IT_과학>IT_과학일반,문화>요리_여행,IT_과학>과학,경제>서비스_쇼핑,경제>외환,미분류,문화>미술_건축,문화>생활,경제>경제일반,국제>아시아,정치>정치일반,경제>국제경제,경제>취업_창업,IT_과학>보안,정치>외교,경제>무역,문화>영화,정치>북한,경제>산업_기업,문화>종교,경제>반도체,문화>학술_문화재,사회>노동_복지,국제>국제일반,IT_과학>인터넷_SNS,국제>러시아
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57


In [ ]:
df.columns

Index(['date', 'title', 'topic', 'keyword', 'input', 'body', 'label'], dtype='object')

In [ ]:
df[df.label==0]

,date,title,topic,keyword,input,body,label
8,20201128,"'온앤오프' 이지아, '데뷔 후 최초' 베일 싸여있던 일상 공개",문화>방송_연예,"온앤오프,이지아,데뷔,베일,일상,공개,28일,방송,tvN,온앤오프,배우,이지아,반전...",이지아 온앤오프 덕후 한경닷컴 김나경 tvn 은하계 ufo 동작 하나하나 외계인설 ...,28일(토) 방송하는 tvN ’온앤오프‘에서는 배우 이지아의 반전 매력 넘치는 일상...,0
10,20201128,"'런닝맨' 김소연, '팬심' 폭발시킨 멤버 정체는?",문화>방송_연예,"런닝맨,김소연,팬심,폭발,멤버,정체,배우,김소연,팬심,고백,29일,방송,SBS,런닝...",김소연 런닝맨 팬심 이상우 멤버들 리액션 반박해 180도 다들 거짓말 펜트하우스 1...,배우 김소연이 수줍은 팬심을 고백한다.\n오는 29일 오후 방송되는 SBS ‘런닝맨...,0
15,20201128,"‘트로트의 민족’ 김소연, 우승 후보→패자부활전 합격할까[M+TV인사이드]",문화>방송_연예,"트로트,민족,김소연,우승,후보,패자부활전,합격,TV인사이드,트로트,민족,유력,우승,...",트로트 박하명 긴장감 김소연 안성준 시청률 신명근 생존자 패자부활전 장명서 나미애 ...,‘트로트의 민족’ 유력 우승 후보 김소연이 패자부활전에 나선 가운데 그의 생존에 관...,0
21,20201128,"'아는 형님' 조병규, 즉석 액션부터 성대모사까지 매력 대방출!",문화>방송_연예,"형님,조병규,즉석,액션,성대모사,매력,대방출,배우,조병규,예능,블루칩,등극,방송,J...",조병규 김세정 눈물 성대모사 유준상 친밀도 만큼 촬영장 웃픈 블루칩 자유자재 전학생...,배우 조병규가 넘치는 끼로 예능 블루칩에 등극했다.\n \n28일 방송되는 JTBC...,0
67,20201128,‘온앤오프’ 이지아 “UFO 출몰 지역을 간 적도 있다”,문화>방송_연예,"온앤오프,이지아,UFO,출몰,지역,28일,방송,tvN,온앤오프,배우,이지아,반전,매...",이지아 온앤오프 덕후 ufo tvn 동작 하나하나 은하계 외계인설 싱어송라이터 놀라...,[헤럴드경제 = 서병기 선임기자]28일(토) 밤 방송되는 tvN ’온앤오프‘에서는 ...,0
...,...,...,...,...,...,...,...
19069,20201201,영화보며 시간 여행 자연 힐링 독서하며 언택트 송년,문화>방송_연예,"영화,여행,자연,힐링,독서,언택트,송년,연말,영화,리틀,포레스트,Little,개봉,...",구멍가게 일본 순천만 미국 곽재구 루나나 해남 목포 원스 혜원 보이저 시간여행 드루...,[연말에 보면 좋은 영화]\n▲리틀 포레스트(Little Forest /감독 임순례...,0
19304,20201201,"[종합] 아이랜드 통해 데뷔한 '엔하이픈' ""7명 전원이 센터급 비주얼"" 연말 신...",문화>방송_연예,"아이랜드,데뷔,엔하이픈,전원,센터,비주얼,연말,신인상,사진,빌리프랩,제공,빅히트엔터...",엔하이픈 아이랜드 빅히트 니키 희승 제이크 제이 타이틀곡 성훈 7명 신인상 팬들 방...,[사진= 빌리프랩 제공] \n \n \n 빅히트엔터테인먼트와 Mnet의 프로젝트 ...,0
19581,20201130,"[이웃사랑] 뇌병변에 유방암까지 온 아내, 홀로 고군분투 중인 남편",문화>방송_연예,"뇌병변,유방암,아내,고군분투,남편,결혼식,뇌병변,장애,남편,4년,병간호,유방암,생활...",주희 영석 결혼식 예주 유방암 중국집 대구은행 병간호 핸드폰 일자리 생활비 매일신문...,지난 27일 달서구 감삼동의 한 주택 2층. 쿵쾅쿵쾅 계단 위로 한 여자아이가 급히...,0
19598,20201130,전효성 다이어트 “3개월만에 허리 26인치→23.5인치”,문화>방송_연예,"3개월,전효,다이어트,허리,인치,23.5인치,가수,전효,다이어트,영상,공개,화제,전...",전효성 전효 3개월 다이어터 필라테스 배달음식 필요성 유튜브 장장 2개월 식이요법 ...,가수 전효성이 다이어트 영상을 공개해 화제다.\n\n\n\n전효성이 30일 오후 자...,0


In [ ]:
df[df.label==10]

,date,title,topic,keyword,input,body,label
61,20201128,"옥천군, 여성폭력 추방 캠페인 실시",사회>여성,"옥천군,캠페인,여성,폭력,추방,캠페인,충북일보,옥천군,추방주간,여성,폭력,추방,주간...",여성폭력 성매매 성폭력 전단지 추방기간 옥천군여성단체협의회원 옥천여자중학교 옥천군 ...,［충북일보］ 옥천군은 27일 '여성폭력 추방주간'을 맞아 옥천여자중학교 앞에서 옥천...,10
103,20201128,"""사지 마비된 동생, 사과 없는 가해자 엄중 처벌해달라"" 피해자 가족 호소",사회>여성,"마비,동생,사과,가해자,처벌,호소,피해자,가족,경남,진주,발생,칼치기,사고,버스,여...",가해자 청원인 피해자 청원 이종기 변호사 진주지원 공소장 고등학교 특례법 졸업식 진...,2019년 12월 경남 진주에서 발생한 '칼치기 사고'로 버스에 타고 있던 고등학교...,10
197,20201128,경남 창원 단란주점 관련 4명 진주 이통장 2명 등 13명 추가 확진,사회>여성,"단란주점,4명,경남,창원,단란주점,진주,통장,확진,추가,경남,신종,코로나바이러스,감...",확진자 50대 진주 창원 경남 사천 단란주점 60대 김해 2명 4명 접촉자 감염경로...,[아시아경제 영남취재본부 주철인 기자] 경남의 신종 코로나바이러스감염증(코로나19)...,10
320,20201128,"""뺨을 베어갈 듯"" 칼바람 막아줄 수분 신상템",사회>여성,"칼바람,수분,기온,칼바람,피부,건조,겨울철,제품,피부,장벽,케어,겨울,바람,피부,보...",저자극 마이크로바이옴 고보습 화장품 활성화 유익균 제니피끄 흡수력 비건 보습력 스킨...,[아시아경제 조유진 기자] 급격히 떨어진 기온과 칼바람으로 피부가 건조해지기 쉬운 ...,10
397,20201128,"비혼 임신, 법과 현실의 간극",사회>여성,"비혼,임신,현실,간극,사유리,방송인,후지타,비혼출산,한국사회,파장,한국,결혼,사람,...",기증자 비혼여성 비혼 생명윤리법 시험관 윤리지침 난임 인공수정 난임부부 한국 나영정...,방송인 후지타 사유리의 비혼출산은 한국사회에 적잖은 파장을 불러왔다. “한국에서는 ...,10
...,...,...,...,...,...,...,...
19191,20201201,"대한적십자사봉사회 괴산지구협의회, 희망 꾸러미 키트 나눔 펼쳐",사회>여성,"대한적십자사봉,사회,괴산지구협의회,키트,희망,꾸러미,대한적십자사봉,사회,괴산,지구,...",코로나19 괴산 괴산지구협의회 키트 김영 취약계층 10여 최준환 적십자봉사회 봉사활...,[충청투데이 김영 기자] 대한적십자사봉사회 괴산지구협의회가 연탄나눔 봉사활동에 이어...,10
19316,20201201,코로나 블루 극단적 선택 여성 급증 스마트폰 대신 햇빛 보세요,사회>여성,"코로나,블루,극단,선택,여성,급증,스마트폰,햇빛,운동,도움,우울증,예방,고위험군,정...",우울증 코로나 50대 재택근무 부산 스마트폰 사망자 생활비 관련자 정신과 좌절감 자...,- 정부 차원 고위험군 서둘러 발굴\n- 상담 제공 등 안전망 관리해야\n\n올들어...,10
19423,20201201,코로나 연말연시 모이거나 미루거나 멈추거나,사회>여성,"코로나,연말연시,풍경,도민,연말맞이,감염증,신종,코로나바이러스,코로나19,재확산,연...",코로나19 창원 파티룸 친구들 확진자 도민들 연말맞이 2단계 대유행 마산 연말연시 ...,신종 코로나바이러스 감염증(코로나19)이 재확산되는 가운데 연말연시를 맞는 도민들의...,10
19537,20201201,화성시 12월부터 사회적 거리두기 단계별 일회용품 사용규제 시행,사회>여성,"화성시,사회,거리,단계,일회용품,사용,규제,시행,화성시,12월,사회,거리,사용규제,...",일회용품 다회용 사용규제 과태료 코로나19 시행령 화성시 그동안 식품접객업소 활용법...,화성시는 12월부터 사회적 거리두기 단계별 일회용품 사용규제를 시행한다고 30일 밝...,10


In [ ]:
NUM_LABELS = max(set(df['label'])) + 1
NUM_LABELS

58

In [ ]:
text_to_label = dict(labels_csv)

In [ ]:
label_to_text = {}
for label in text_to_label:
    label_to_text[int(text_to_label[label])] = label

In [ ]:
label_to_text

{0: '문화>방송_연예',
 1: '국제>중국',
 2: '사회>사회일반',
 3: '정치>국회_정당',
 4: '사회>교육_시험',
 5: 'IT_과학>모바일',
 6: '정치>행정_자치',
 7: '사회>사건_사고',
 8: '문화>문화일반',
 9: '정치>청와대',
 10: '사회>여성',
 11: '국제>유럽_EU',
 12: '경제>자원',
 13: '국제>미국_북미',
 14: '사회>날씨',
 15: '사회>의료_건강',
 16: 'IT_과학>콘텐츠',
 17: '경제>증권_증시',
 18: '정치>선거',
 19: '국제>일본',
 20: '사회>미디어',
 21: '문화>음악',
 22: '국제>중남미',
 23: '국제>중동_아프리카',
 24: '경제>유통',
 25: '문화>출판',
 26: '경제>자동차',
 27: '사회>환경',
 28: '경제>금융_재테크',
 29: '사회>장애인',
 30: '문화>전시_공연',
 31: '경제>부동산',
 32: 'IT_과학>IT_과학일반',
 33: '문화>요리_여행',
 34: 'IT_과학>과학',
 35: '경제>서비스_쇼핑',
 36: '경제>외환',
 37: '미분류',
 38: '문화>미술_건축',
 39: '문화>생활',
 40: '경제>경제일반',
 41: '국제>아시아',
 42: '정치>정치일반',
 43: '경제>국제경제',
 44: '경제>취업_창업',
 45: 'IT_과학>보안',
 46: '정치>외교',
 47: '경제>무역',
 48: '문화>영화',
 49: '정치>북한',
 50: '경제>산업_기업',
 51: '문화>종교',
 52: '경제>반도체',
 53: '문화>학술_문화재',
 54: '사회>노동_복지',
 55: '국제>국제일반',
 56: 'IT_과학>인터넷_SNS',
 57: '국제>러시아'}

In [ ]:
train, test = train, test = train_test_split(df, test_size=0.1)

print(train.shape)
print(test.shape)

(43631, 7)
(4848, 7)


In [ ]:
train.head(10)

,date,title,topic,keyword,input,body,label
18499,20201126,"영동군, 65세 이상 노인 대상포진 무료예방접종 실시",사회>의료_건강,"영동군,65세,충북,영동군,30일,65세,노인들,노인,대상,예방접종,대상,포진,무료...",대상포진 영동군 65세 예방접종 대상자 안내문 발생률 코로나19 노인들 대상자들 영...,충북 영동군은 2020년 11월 30일부터 만 65세 이상 노인들을 대상으로 대상포...,15
17977,20201126,"'유키스 출신' 일라이, 지연수와 이혼 발표 ""아버지로 최선 다할 것""",문화>방송_연예,"유키스,출신,일라이,지연수,이혼,아버지,이혼,최선,일라이,그룹,유키스,아내,지연수,...",일라이 유키스 지연수 혼인신고 그동안 사진제공 연상연하 돌잔치 결혼식 미국 앞날 한...,그룹 유키스 일라이가 아내 지연수와 이혼을 발표했다.\n\n26일 일라이는 자신의 ...,0
18429,20201201,"스파크플러스, 포스트 코로나 '뉴노멀 오피스' 강남4호점 열어",경제>경제일반,"스파크플러스,포스트,코로나,뉴노멀,오피스,강남,호점,열어,스파크플러스,감염증,신종,...",강남 서울 스파크플러스 공유오피스 강남역 강남4호점 뉴노멀 호점 입주사 강남4호 열...,스파크플러스가 신종 코로나바이러스 감염증(코로나19)시대 이후 근무 환경 변화를 고...,40
3104,20191128,"LG전자, 새 사령탑에 권봉석 사장 선임",IT_과학>모바일,"LG전자,사령탑,선임,권봉석,사장,HE사업본부장,MC,HE사업본부장,권봉석,LG,전...",lg전자 권봉석 부사장 부회장 전략가 서울대학교 mc lg 본부장 사령탑 사업부장 ...,[디지털타임스 이미정 기자] 권봉석 LG전자 MC HE사업본부장이 조성진 부회장의 ...,5
5414,20201204,"""역사 전쟁 제대로 하라""...日 자민당 의원들, 정부에 결의문",국제>일본,"역사,전쟁,의원들,자민당,의원,정부,결의문,앵커,의원들,일본,자민당,의원,일본,정부...",일본 중국 베를린 모테기 센카쿠 미테구 소녀상 결의문 자민당 도쿄 이경아 결의안 독...,[앵커]\n일본 자민당 의원들이 일본 정부에 이른바 '역사 전쟁'에 더 강하게 나설...,19
7636,20201203,SK그룹 정기 임원인사 박정호 유정준 부회장 승진,경제>산업_기업,"임원인사,SK,그룹,정기,임원,인사,박정호,유정준,부회장,승진,사장,박정호,SK,텔...",sk 위원회 부회장 위원장 박정호 전문가 하이닉스 관계사 유정준 수펙스추구협의회 s...,[디지털타임스 김위수 기자] 박정호 SK텔레콤 사장과 유정준 SK E&S 사장이 각...,50
14996,20201120,"네이버 악플차단 AI는 안다 ""단어 '쓰레기'는 OK, 욕 '쓰레기'는 NO""",IT_과학>인터넷_SNS,"AI,네이버,악플,차단,단어,쓰레기,OK,쓰레기,네이버,댓글,서비스,시작,공론,역할...",악플 네이버 클린봇 ai 바이브 아시아경제 성폭력 이규호 건설적 인공지능 띄어쓰기꼼...,[아시아경제 부애리 기자] 네이버가 댓글 서비스를 시작한 지 17년째. '공론의 장...,56
11543,20201202,"'디지털' 앞에 '순혈' 없다 銀, 혁신조직 만들고 외부인재 영입 박차(종합)",경제>금융_재테크,"디지털,순혈,혁신조직,박차,외부,인재,영입,종합,디지털,금융,강화,사활,시중은행들,...",전문가 사업부 부행장 본부장 ai 삼성 농협은행 우리은행 빅데이터 국민은행 은행장 ...,[아시아경제 조강욱 기자] '디지털 금융' 강화에 사활을 건 시중은행들이 적극적인 ...,28
15287,20201126,신논현역 인근 제주 근고기 맛집 ‘육덕식당’,문화>문화일반,"신논현역,인근,제주,근고기,육덕식당,신논현역,출구,인근,간판,상호,로고,복고풍,추억...",근고기 제주 육덕식당 신논현역 덩어리째 600g 강남구 연탄구이 연탄불 제주도 돼지...,"신논현역 3번 출구 인근에 간판, 상호, 로고 등이 복고풍이라 추억을 소환하는 제주...",8
12309,20201204,"​2030대 청약통장 보유율 85% 육박하지만 ""분양은 잘 몰라""",경제>부동산,"육박,청약,통장,보유,85%,분양,다방,관심,분양,정보,조사,시행,사진,다방,제공,...",2030대 이용자 청약통장 부동산 행복주택 응답자 설문조사,[사진=다방 제공] \n \n 부동산에 대한 관심이 증가하면서 2030대 10명중...,31


<br>
<br>

# **Preprocessing - TRAINING SET**

In [ ]:
# 문장 추출
sentences = train['input'].values
sentences[:10]

array(['대상포진 영동군 65세 예방접종 대상자 안내문 발생률 코로나19 노인들 대상자들 영동읍 대한민국 보건지소 수두바이러스 신경절 활성화 신경괴사 면역기능 노후생활 면역력 보건소',
       '일라이 유키스 지연수 혼인신고 그동안 사진제공 연상연하 돌잔치 결혼식 미국 앞날 한국 sns 아들 이혼 부부 엄마 아내 최선 아버지 그룹 나이 차이 각자 결국 응원 극복 기도 축하 정식 사진 진심 화제 혼인 신고',
       '강남 서울 스파크플러스 공유오피스 강남역 강남4호점 뉴노멀 호점 입주사 강남4호 열어 통유리 코로나바이러스 ir 목진건 방문객 출입관리 멀티비전 코로나',
       'lg전자 권봉석 부사장 부회장 전략가 서울대학교 mc lg 본부장 사령탑 사업부장 경영자 경영방식 의사결정 수익구조 성과주의 모니터사업부장 시너지팀장 산업공학 연구개발 팀장 잠재력 그룹장 사업성 사업전반 상품기획 사업경쟁력 경쟁력 미래준비 최고경영자 조성진 5명',
       '일본 중국 베를린 모테기 센카쿠 미테구 소녀상 결의문 자민당 도쿄 이경아 결의안 독일 외교부장 의원들',
       'sk 위원회 부회장 위원장 박정호 전문가 하이닉스 관계사 유정준 수펙스추구협의회 sk그룹 소위원회 염용섭 추형욱 협의회 거버넌스위원회 김준 이석희 코로나',
       '악플 네이버 클린봇 ai 바이브 아시아경제 성폭력 이규호 건설적 인공지능 띄어쓰기꼼수 웹툰',
       '전문가 사업부 부행장 본부장 ai 삼성 농협은행 우리은행 빅데이터 국민은행 은행장 추진단 ict dt',
       '근고기 제주 육덕식당 신논현역 덩어리째 600g 강남구 연탄구이 연탄불 제주도 돼지고기 포차 영업시간 논현동 복고풍 환상궁합 멸치젓 된장찌개 강남대로 공깃밥 청정지역 오겹살 껍데기살 직원들 목살구이 김치찌개 1인',
       '2030대 이용자 청약통장 부동산 행복주택 응답자 설문조사'], dtype=object)

In [ ]:
# 라벨 추출
labels = list(map(str, train['label'].values))
labels[:10]

['15', '0', '40', '5', '19', '50', '56', '28', '8', '31']

In [ ]:
train_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

<br>
<br>

# **Preprocessing - TEST SET**

In [ ]:
# 문장 추출
sentences = test['input'].values
sentences[:10]

array(['소비자 장병현 인천시 1만 복지관 인천시노인종합복지관협회 코로나19 일자리 일자리경제과장 급성장 노인들 이용법 정보 노인 디지털 어르신 행사 회원 소비 꾸러미 관내 피해 생활 사례 효율적 진행 효율 대응 경제 향상 과장',
       '올마이티캠퍼스 서울대 학생들 교육서비스 독서실 공부법 학습관리서비스 학습관리 멘토 과목별 전문가 롯데백화점 한티역 정규수업 시험공부 수학수업 3박자',
       '시험장 서울 학년도 동작구 숭실대학교 학부모 자녀 논술 격려 수학 능력 대학 시험 수험',
       '메가엠디 지분변동 특별관계자 관계자 관계인 특수관계인 거래량 보고서 거래일 주식등 대량보유상황보고서 메가스터디 지분율 지분 공시 보유 변동 주식 직전 대비 일명 특별 특수 포함 개인 매입 매각 법인 이동 추가 상황 대량 종가 전문 하락 마감 기업 상장 회사',
       '확진자 코로나19 수험생 시험실 지원자 감독관 시험장 학년도 자가격리자 교육부 49만',
       '명신고 이사장 하춘욱 씨앤투스성진 코스닥 코로나19 明德 명덕 사람들 후배들 온몸 明德新民 명신고등학교 교직원 이현철 학교생활 청정기 생산품 新民 명덕신민 교직원들 진공청소기 보건용 건학이념 이사장님 졸업생 선배님 공기청정기 눈앞 자동차 예비심사 후배사랑 김성우',
       '코로나19 인종차별 라틴계 라틴 그동안 cnn 미국 합병증 커닝엄',
       '협력사 lg전자 혁신활동 경쟁력 포상금 6억 자동화 생산라인 20억 사업장 내년도 생산성 상생성과 혁신성과 제조경쟁력 코로나19 제품경쟁력 lg 5000만 생산공정 우수 전자 생산 규모 혁신 성과 편리 레터 상생 선정 활동 자동',
       '코로나19 당선인 미국 바이든 행정부 인수위 조지 권위자 코로나 도널드 앤서니 감염병',
       '재택근무 생산성 한국노동연구원 코로나19 개발도상국 노동자 에릭 국제노동기구 미국 코로나 원격근무 배규식 스탠퍼드대학교 서울대 건국대 김근주 장지연 제이 조용만 선임연구위

In [ ]:
# 라벨 추출
labels = list(map(str, test['label'].values))
labels[:10]

['35', '4', '4', '31', '4', '40', '15', '34', '13', '10']

In [ ]:
test_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

In [ ]:
dataset_train = train_dataset.values
dataset_test = test_dataset.values

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        
        for i in dataset:
            try:
                self.sentences.append(transform([i[sent_idx]]))
                self.labels.append(np.int32(i[label_idx]))
            except Exception as e:
                print(e)
                print(len(self.sentences), len(self.labels))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

Argument 'text' has incorrect type (expected str, got float)
7436 7436
Argument 'text' has incorrect type (expected str, got float)
15321 15321
Argument 'text' has incorrect type (expected str, got float)
21959 21959
Argument 'text' has incorrect type (expected str, got float)
23043 23043
Argument 'text' has incorrect type (expected str, got float)
27053 27053
Argument 'text' has incorrect type (expected str, got float)
28719 28719
Argument 'text' has incorrect type (expected str, got float)
35454 35454


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_LABELS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.25).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
from time import gmtime, strftime
START_TIME = strftime("%Y_%m_%d_%H_%M_%S", gmtime())

In [ ]:
SAVE_PATH = BASE_PATH+"/bert_news_model_{}.pth".format(START_TIME)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    torch.save(model.state_dict(), SAVE_PATH)
    print("Model saved at {}".format(SAVE_PATH))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 4.07710599899292 train acc 0.03125
epoch 1 batch id 201 loss 3.92838716506958 train acc 0.04329912935323383
epoch 1 batch id 401 loss 3.147287130355835 train acc 0.1345074812967581
epoch 1 batch id 601 loss 2.8062405586242676 train acc 0.22153182196339435

epoch 1 train acc 0.24761271994134895
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_17_15_55_09.pth



epoch 1 test acc 0.4641584429824561


epoch 2 batch id 1 loss 2.3890326023101807 train acc 0.46875


KeyboardInterrupt: ignored